In [136]:
using LinearAlgebra, LineSearches, Optim

# ME 617 HW 1

_Note: This source code was implemented in Julia. This notebook requires a Julia 1.7.x kernel as well as an environment that includes the necessary dependencies. See the [repo](https://github.com/camirmas/DesignAutomation) for full implementations, testing, and dependency information. All relevant code has been copied into this notebook, so no importing of individual modules is necessary._

All problems involve the following objective function:

$f(x_1, x_2) = (2x_2-3x_1^2)^4+(5-2x_1)^2$

And use the starting point:

$(x_1, x_2) = (0, 0)$

In [137]:
fn(x) = (2*x[2]-3*x[1]^2)^4 + (5-2*x[1])^2
x0 = [0., 0.]

2-element Vector{Float64}:
 0.0
 0.0

For reference, let's see what the answer to this problem is according to a popular optimization library, [Optim](https://github.com/JuliaNLSolvers/Optim.jl):

In [138]:
res = optimize(fn, x0)

 * Status: success

 * Candidate solution
    Final objective value:     1.065787e-09

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    55
    f(x) calls:    107


In [139]:
res.minimizer

2-element Vector{Float64}:
 2.5000162227450113
 9.37607248831831

## 1. CCS

Description: 

Solve the problem with Cyclic Coordinate Search for 12 iterations. Solve (a) with acceleration steps and compare to (b) without the acceleration step. For (a), you would search $e_1$, then $e_2$, then the acceleration, four times. For (b), you would search $e_1$, $e_2$, for six times). Recall that this method requires a line search like Golden Section...

#### a)

This version of the algorithm includes acceleration, and was initially implemented using convergence criteria (commented out because otherwise it converges before reaching 12 iterations). The fully tested algorithm can be found in the [repo](https://github.com/camirmas/DesignAutomation/blob/main/src/ccs.jl) with accompanying [tests](https://github.com/camirmas/DesignAutomation/blob/main/test/test_ccs.jl). Note that this function actually reaches convergence well before 12 iterations (it appears to be done after 4).

In [140]:
struct Response
    minimizer
    minimum::AbstractFloat
    iterations::Int
    converged::Bool
end

In [141]:
"""
Implements the Cyclic Coordinate Search optimization method. Uses a Hager-Zhang
linesearch to determine step size.
"""
function ccs_a(fn, x0; ϵ=.001, max_iter=100)
    algo_hz = Newton(linesearch = HagerZhang())
    i = 1
    k = 0
    n = length(x0)
    results = [(x0, fn(x0))]
    e_hat = Matrix(1I, n, n)

    while k < max_iter
        x, _ = results[end]

        if i == n + 1
            d_k = x - results[k-n+1][1]
            i = 1
        else
            d_k = e_hat[:, i]
        end

        α_fn(α) = fn(x + α .* d_k)
        res = Optim.optimize(α_fn, x, method=algo_hz)

        α = res.minimizer
        x_new = x + α .* d_k
        f_new = fn(x_new)
        push!(results, (x_new, f_new))

        k += 1
        i += 1

#         if norm(x_new-x) < ϵ
#             return Response(x_new, f_new, k, true)
#         end
    end

    println("Maximum iterations reached.")
    x, fx = results[end]
    return Response(x, fx, k, false)
end


ccs_a

In [142]:
res = ccs_a(fn, x0; max_iter=12)
res

Maximum iterations reached.


Response([2.500000005615144, 9.374999993584302], 1.2611936431358006e-16, 12, false)

In [143]:
res.minimizer

2-element Vector{Float64}:
 2.500000005615144
 9.374999993584302

In [144]:
res.minimum

1.2611936431358006e-16

#### b)

This function has been modified to remove the acceleration step. Note that this function does not actually converge in 12 iterations. This suggests that acceleration strongly influences the speed at which CCS converges, at least for objective functions resembling this one.

In [145]:
"""
Implements the Cyclic Coordinate Search optimization method. Uses a Hager-Zhang
linesearch to determine step size.
"""
function ccs_b(fn, x0; ϵ=.001, max_iter=100)
    algo_hz = Newton(linesearch = HagerZhang())
    i = 1
    k = 0
    n = length(x0)
    results = [(x0, fn(x0))]
    e_hat = Matrix(1I, n, n)

    while k < max_iter
        x, _ = results[end]

        if i > n
            i = 1
        end
        
        d_k = e_hat[:, i]

        α_fn(α) = fn(x + α .* d_k)
        res = Optim.optimize(α_fn, x, method=algo_hz)

        α = res.minimizer
        x_new = x + α .* d_k
        f_new = fn(x_new)
        push!(results, (x_new, f_new))

        k += 1
        i += 1

#         if norm(x_new-x) < ϵ
#             return Response(x_new, f_new, k, true)
#         end
    end

    println("Maximum iterations reached.")
    x, fx = results[end]
    return Response(x, fx, k, false)
end

ccs_b

In [146]:
res = ccs_b(fn, x0; max_iter=12)
res

Maximum iterations reached.


Response([1.2774343915191781, 2.4477579276687598], 5.978666668160328, 12, false)

In [147]:
res.minimizer

2-element Vector{Float64}:
 1.2774343915191781
 2.4477579276687598

In [148]:
res.minimum

5.978666668160328

#### Conclusion

Overall, these two examples emphasize the strength of acceleration steps, at least in this scenario. In comparing this implementation of CCS against other algorithms such as Hooke & Jeeves, it is useful to remember that the number of iterations performed in the 1-D line search is not encapsulated in the result. Even though the accelerated CCS completed in 4 "iterations," the total number of function calls is higher.

## 2. Hooke & Jeeves

Description: Solve the problem using Hooke and Jeeves for 24 function evaluations. Set r = 0.5 and h0 = 0.5.

This algorithm was initially implemented using convergence criteria, which don't appear to be met for the number of iterations requested for this problem. The fully tested algorithm can be found in the [repo](https://github.com/camirmas/DesignAutomation/blob/main/src/hooke_jeeves.jl) with accompanying [tests](https://github.com/camirmas/DesignAutomation/blob/main/test/test_hooke_jeeves.jl).

In [149]:
"""
Implements the Hooke & Jeeves direct search method.

Arguments:
    fn: objective function
    x0: starting point
    h0: starting step size
    h_min: minimum step size
    r: reduction factor for step size

Returns:
    `Response` struct if successful, else `nothing`
"""
function hooke_jeeves(fn, x0; h0=1/10, h_min=.01, r=1/2, max_iter=1000)
    x = copy(x0)
    n = length(x)
    e_hat = Matrix(1I, n, n)
    k = 0
    h = h0
    x_b = x
    fx = fn(x)
    results = [(x, fx)]

    while k < max_iter
        for i = 1:n
            x, fx = results[end]
            # try searching in each linearly independent direction
            x_new = x + h * e_hat[:, i]
            f_new = fn(x_new)

            if f_new <= fx
                k += 1
                push!(results, (x_new, f_new))
            else
                x_new = x - h * e_hat[:, i]
                f_new = fn(x_new)

                if f_new <= fx
                    k += 1
                    push!(results, (x_new, f_new))
                end
            end
        end

        x, fx = results[end]
        # if the base hasn't changed, reduce step size
        if x_b == x
            h *= r

            # convergence reached
            if h < h_min
                return Response(x, fx, k, true)
            end
        end

        # perform acceleration step
        x_new = 2x - x_b
        x_b = x
        f_new = fn(x_new)

        if f_new <= fx
            k += 1
            push!(results, (x_new, f_new))
        end
    end

    println("Maximum iterations reached.")
    x, fx = results[end]
    return Response(x, fx, k, false)
end

hooke_jeeves

In [150]:
res = hooke_jeeves(fn, x0; r=.5, h0=.5, max_iter=24)

Maximum iterations reached.


Response([1.5, 3.25], 4.00390625, 24, false)

This is not a very promising result after 24 iterations, given that we know the true answer. For reference, if we let the algorithm run to convergence (`r=.5`, `h0=.1`, `hmin=.01`), we get the following results, which appear consistent with the true answer:

In [151]:
res = hooke_jeeves(fn, x0)

Response([2.4999999999999996, 9.37500000000003], 7.888609052210118e-31, 99, true)

While the default knobs for this function result in convergence after 99 iterations, it is possible that an improved choice of knob values would yield the same result in less iterations.

## 3. Discussion

Description: Discuss differences in 1 & 2 using the criteria presented in ME517 (see slide 4 of Section5 slides). You can cite evidence from your results above, from challenges in writing the code, but also extrapolate to challenges in different problems, and iteration limits.

#### Robustness

Similar to the discussion about ease of use, CCS appears as robust as, if not more robust than, H&J, partially due to the fact that, at least in this implementation, H&J depends on the user choosing acceptable values for step sizes and reduction for the given sample space. A potential pitfall of this approach would involve the algorithm missing a minimum point (e.g. the step size is too large), or, as discussed in the textbook, taking too many small steps (see Efficiency). CCS, on the other hand, uses a robust, but potentially less efficient line search to determine step size at each iteration.

#### Generality

Neither algorithm imposes any particular restrictions on the objective function in a way that might cause issues. Both are intended to search the space as-is, as opposed to methods that attempt impose penalty functions, for example (though it should be noted that neither implementation is designed to handle constraints).

#### Accuracy

Both methods appear able to achieve precise and accurate results for minimum points for the types of problems investigated thus far in the course (e.g. Rosenbrock's banana). See [tests](https://github.com/camirmas/DesignAutomation/tree/main/test) for more examples of these algorithms performing against additional objective functions.

#### Ease of use

While both algorithms emphasize a simple, intuitive approach to optimization, and are relatively straightforward to implement, H&J appears more difficult for an end user, primarily because it has more tunable coefficients than Cyclic Coordinate Search. With H&J, small changes to `r`, `h0`, and `hmin` may have large influences on the algorithm's convergence, and may in fact cause problems with convergence (reducing too slowly or too quickly and/or defining step sizes that are too big or too small).

#### Efficiency

Comparing efficiency between CCS and H&J requires a number of considerations. Primarily, CCS and H&J are measuring "iterations" differently, namely that the 1-D linesearch in CCS is not included in the iteration counter, though it performs a non-negligible number of iterations of its own. As such, iteration limits for each algorithm should be set and considered differently. Additionally, as discussed in class, in the textbook, and in this assignment, H&J's performance is highly dependent on the user-provided knob settings. Poor settings may cause an unnecessarily high number of iterations and/or poor results, while informed ones may lead to a more efficient solution than that of CCS.